In [ ]:
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import matplotlib

import h5py
import plotly.graph_objs as go
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML
plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))
import pandas as pd
import sys
import os
import re

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
#Getting a values 
# folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5\20250611 LSU eps 9"
folder_path = r"H:\phd stuff\tidy3d\structures\LSU H5\20250611 LSU eps 11p56"
a_values = {}
for f,filename in enumerate(os.listdir(folder_path)):
    print(filename)
    if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
        continue 
    if os.path.isfile(os.path.join(folder_path, filename)):
        file=os.path.join(folder_path, filename)
        ff= re.search(r'ff_0.\d+', filename).group(0)
        key=float(f"0.{ff[5:]}")
        if key not in a_values:
            a_values[key] = {}
        a_values[key]["value"]=get_a_from_h5_eps(file,14.3)[0]
                


In [ ]:
#Convergence

data = {}
for path_direction in [
                 
                  rf"H:\phd stuff\tidy3d\data\20250611 different ff_perm_11.56\z_incidence",
                  rf"H:\phd stuff\tidy3d\data\20250611 different ff_perm_9.00\z_incidence"
                      
                       ]:

      folder_path = f"{path_direction}"
      perm= float(re.search(r'perm_(\d+(?:\.\d+)?)', folder_path).group(1))
      
      if perm not in data:
          data[perm] = {}
      for i,filename in enumerate(natsorted(os.listdir(folder_path))):
            
           
            if not Path(filename).suffix==".txt":
                  continue
            ff= re.search(r'ff_0.\d+', filename).group(0)
            ff_value=float(f"0.{ff[5:]}")
            
            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file)
                try:
                  sim_data0 =  structure_1.sim_data0
                  transmission0_right = sim_data0['flux1'].flux
                  transmission0_left = sim_data0['flux2'].flux
                except:
                  print("No Reference")

                sim_data = structure_1.sim_data
                key = ff_value
                if key not in data[perm]:
                    data[perm][key] = {}
                data[perm][key]["right"]=sim_data['flux1'].flux.values
                data[perm][key]["left"]=sim_data['flux2'].flux.values

monitor_lambdas = (td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs))

In [ ]:
perm_vals_sorted=np.array(sorted(set(x for x in data.keys())))
ff_vals_sorted=np.array(sorted(set(x for x in data[perm_vals_sorted[0]].keys())))

In [ ]:
font = {
        'weight' : 'bold',
        'size'   : 20}

matplotlib.rc('font', **font)

fig,ax = plt.subplots(2,2,figsize=(25, 10))
colors = plt.cm.gist_rainbow(np.linspace(0, 1, len(ff_vals_sorted)))

for l,perm_val in enumerate(perm_vals_sorted):
 for i,ff in enumerate(ff_vals_sorted): 
    normalized_transmission = data[perm_val][ff]["right"]/transmission0_right
    ax[0,l].plot(monitor_lambdas*0.8,normalized_transmission, label=rf"ff={ff}; $a^0_{{eff}}={a_values[ff]["value"]*0.8:.4f} \mu m$",color=colors[i])
    ax[1,l].plot(monitor_lambdas*0.8*0.93,normalized_transmission, label=fr"ff={ff}; $a_{{eff}}={a_values[ff]["value"]*0.8*0.93:.4f} \mu m$",color=colors[i])
    ax[0,l].set_title(rf"Digital Template n={np.sqrt(perm_val)}")
    ax[1,l].set_title(rf"Shrinkage s=0.93 n={np.sqrt(perm_val)}")
    
# Set common labels
for a in ax.flat:
    a.set_xlabel(r'$\lambda [\mu m]$')
    a.set_ylabel('Transmission [%]')
    a.set_yscale("log")
    a.set_xlim(2,8)
    a.grid(True, which="both")


# Adjust space to add legend at the right
fig.subplots_adjust(right=0.8)
fig.subplots_adjust(top=1.4)
# Create legend in an empty space on the right
legend_ax = fig.add_axes([0.82, 0.79, 0.15, 0.7])  # [left, bottom, width, height]
legend_ax.axis('off')  # Hide the axis
legend_ax2 = fig.add_axes([0.82, 0.09, 0.15, 0.7])  # [left, bottom, width, height]
legend_ax2.axis('off')  # Hide the axis
# Create a legend from one of the axes (any one will do since all share labels)
handles, labels = ax[0, 0].get_legend_handles_labels()
handles2, labels2 = ax[1, 0].get_legend_handles_labels()
legend_ax.legend(handles, labels, loc='center left', fontsize=16, frameon=True, facecolor='white')
legend_ax2.legend(handles2, labels2, loc='center left', fontsize=16, frameon=True, facecolor='white')

plt.show()


In [ ]:
font = {
        'weight' : 'bold',
        'size'   : 20}

matplotlib.rc('font', **font)


for i,ff in enumerate(ff_vals_sorted): 
   
    plt.figure(figsize=(20, 10))
    normalized_transmission_right = data[ff]["right"]/transmission0_right
    normalized_transmission_left = data[ff]["left"]/transmission0_left
    plt.scatter(monitor_lambdas,normalized_transmission_left, label="Entry")
    plt.scatter(monitor_lambdas,normalized_transmission_right, label="Exit")
    plt.xlabel(rf'$\lambda$')
    plt.ylabel('Transmission [%]')
    plt.title(f"ff={ff}")
    plt.legend()
    plt.yscale("log")
    plt.grid(True,which="both")
    plt.show()
